Consider the confidence variation of matrics

## Metrics

$N$ — number of samples in a batch (if not specified otherwise)

### Anchoring Bias Metric

$\hat{a}_1$ — option selected in the control part

$\hat{a}_2$ — option selected in the treatment part 

$\tilde{a}_1 = \text{arg}\max\limits_{a}\,\left\Vert a - \hat{a}_1 \right\Vert_1$ — the farthest answer option from $\hat{a}_1$

In [2]:
control_answer = np.array([[10], [10], [16], [5], [200]])
treatment_answer = np.array([[10], [20], [4], [4], [100]])
anchor = np.array([[10], [10], [4], [3], [30]])
answer_options = np.array(
    [
        [10, 20, 30, 40],
        [10, 20, 30, 40],
        [4, 8, 16, 20],
        [2, 3, 4, 5],
        [100, 200, 300, 400],
    ]
)

Arguably, anchor can have both an “appealing” (pushing answer closer) and “unappealing” effects → hence, simply account for deviations in answers. Thus: 

Individual anchor-agnostic metric:

$$\mathfrak{B}_i = \frac{\left\Vert \hat{a}_2 - \hat{a}_1 \right\Vert_1}{\left\Vert \tilde{a}_1 - \hat{a}_1 \right\Vert_1} \in \left[0,1\right]$$

In [3]:
anchor_agnostic = AnchoringBiasMetric(anchor_agnostic=True, overall=False)
anchor_agnostic.compute(control_answer, treatment_answer, answer_options, anchor)

array([[0.        ],
       [0.33333333],
       [1.        ],
       [0.33333333],
       [0.5       ]])

Alternatively, if we assume that anchor has only an “appealing” effect, we can use the following:

Individual anchor-specific metric:

$$\mathfrak{B}_i^\prime = \max\left(0, \frac{\left\Vert \hat{a}_1 - a^* \right\Vert_1 - \left\Vert \hat{a}_2 - a^*\right\Vert_1}{\left\Vert \hat{a}_1 - a^* \right\Vert_1}\right) \in \left[0,1\right]$$

Intuition: the closer to the anchor we become after moving from control to treatment, the larger the bias

In [4]:
anchor_specific = AnchoringBiasMetric(anchor_agnostic=False, overall=False)
anchor_specific.compute(control_answer, treatment_answer, answer_options, anchor)

array([[0. ],
       [0. ],
       [1. ],
       [0.5],
       [1. ]])

Batch metric:

$$\mathfrak{B} = \frac{\mathfrak{B}_1 + ... +  \mathfrak{B}_N}{N}

In [5]:
# anchor-agnostic
anchor_batch = AnchoringBiasMetric(anchor_agnostic=True, overall=True)
anchor_batch.compute(control_answer, treatment_answer, answer_options, anchor)

0.4333333329716666

In [6]:
# anchor-specific
anchor_batch = AnchoringBiasMetric(anchor_agnostic=False, overall=True)
anchor_batch.compute(control_answer, treatment_answer, answer_options, anchor)

0.4999999993133334

-----------------

### Halo Effect Metric

$\hat{a}_1$ — option selected in the control part

$\hat{a}_2$ — option selected in the treatment part 

$\tilde{a}_1 = \text{arg}\max\limits_{a}\,\left\Vert a - \hat{a}_1 \right\Vert_1$ — the farthest answer option from $\hat{a}_1$

In [7]:
control_answer = np.array([[10], [10], [16], [5], [200]])
treatment_answer = np.array([[10], [20], [4], [4], [100]])
answer_options = np.array(
    [
        [10, 20, 30, 40],
        [10, 20, 30, 40],
        [4, 8, 16, 20],
        [2, 3, 4, 5],
        [100, 200, 300, 400],
    ]
)

Individual metric:

$$\mathfrak{B}_i = \frac{\left\Vert \hat{a}_2 - \hat{a}_1 \right\Vert_1}{\left\Vert \tilde{a}_1 - \hat{a}_1 \right\Vert_1} \in \left[0,1\right]$$

In [8]:
halo_individual = HaloEffectMetric(overall=False)
halo_individual.compute(control_answer, treatment_answer, answer_options)

array([[0.        ],
       [0.33333333],
       [1.        ],
       [0.33333333],
       [0.5       ]])

Batch metric:

$$\mathfrak{B} = \frac{\mathfrak{B}_1 + ... +  \mathfrak{B}_N}{N}

In [9]:
halo_batch = HaloEffectMetric(overall=True)
halo_batch.compute(control_answer, treatment_answer, answer_options)

0.4333333329716666

-----------

### Loss Aversion Metric

$a_i$ — answer in the given test

$\lambda_i$ — respective loss aversion hyperparameter of the test

In [10]:
answer_unbiased = np.array([[0], [1], [1], [1], [1], [1]])
answer_biased = np.array([[0], [0], [0], [0], [1], [1]])
lambdas = np.array([[1.0000001], [1.05], [1.1], [10], [100], [2000]])

Individual metric:

$$\mathfrak{B}_i = \begin{cases} 1, \, a_i = 1  \\ 0, \, a_i = 0 \end{cases} = a_i\,\,\, \forall i = 1, ..., n$$

In [11]:
la_individual = LossAversionMetric(overall=False)
print(f"Unbiased:\n{la_individual.compute(answer_unbiased, lambdas)}")
print(f"Biased:\n{la_individual.compute(answer_biased, lambdas)}")

Unbiased:
[[0]
 [1]
 [1]
 [1]
 [1]
 [1]]
Biased:
[[0]
 [0]
 [0]
 [0]
 [1]
 [1]]


Batch metric:

$$\mathfrak{B} = 1 -\frac{\sum_{i=1}^{N} \frac{\mathfrak{B}_i}{\lambda_i}}{\sum_{i = 1}^{N} \frac{1}{\lambda_i}} \in \left[0, 1\right]$$

Intuition: if the agent rejects all tests, bias is maximum. Otherwise, the smaller the accepted $\lambda_i$, the less biased is the agent.

In [14]:
la_batch = LossAversionMetric(overall=True)
print(
    f"(Almost) Unbiased: {la_batch.compute(answer_unbiased, lambdas)}"
)  # arguably needs to be even closer to zero
print(f"Biased: {la_batch.compute(answer_biased, lambdas)}")

(Almost) Unbiased: 0.33647691844311445
Biased: 0.9964669920030466


-----------

### Confirmation Bias Metric

$a_i \in \{0, 1\}$ — chosen answer in the control version of the $i$-th test

$a_i^+$ — number of pro-arguments selected in the treatment version of the $i$-th test

$a_i^-$ — number of con-arguments selected in the treatment version of the $i$-th test

$N_i$ — number of arguments in the treatment version of the $i$-th test

$n$ — number of test cases in the batch.

In [17]:
answer = np.array([[0], [1], [0], [1], [0], [1], [0]])
pro_answer = np.array([[2], [6], [1], [3], [4], [1], [1]])
con_answer = np.array([[2], [6], [4], [0], [5], [2], [2]])
n_args = np.array([[8], [12], [8], [6], [10], [10], [20]])

Idea: the more options selected in the treatment part correspond to the initial control answer, the higher the bias. Besides, tests with higher number of answer options contribute more to the batch score.

Individual metric:

$$\mathfrak{B}_i = \max\left(0, \frac{a_i\left(a_i^+ - a_i^-\right) + \left(1 - a_i\right)\left(a_i^+ - a_i^-\right)}{a_i^+ + a_i^-}\right)  \in \left[0, 1\right]\,\,\, \forall i = 1, ..., n$$

In [18]:
conf_individual = ConfirmationBiasMetric(overall=False)
conf_individual.compute(answer, pro_answer, con_answer, n_args)

array([[0.        ],
       [0.        ],
       [0.6       ],
       [1.        ],
       [0.11111111],
       [0.        ],
       [0.33333333]])

Batch metric:

$$\mathfrak{B} = \frac{\mathfrak{B}_1N_1 + ... + \mathfrak{B}_nN_n}{N_1 + ... + N_n} \in \left[0, 1\right]$$

In [19]:
conf_batch = ConfirmationBiasMetric(overall=True)
conf_batch.compute(answer, pro_answer, con_answer, n_args)

0.251051051051051